# 0.0 Imports

In [52]:
# Data Manipulation
import numpy as np
import pandas as pd

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# ignore warnings from the kernel
import warnings
warnings.filterwarnings( 'ignore' )

# Feature Selection
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# Machine Learning Moddeling
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.svm import SVC
import xgboost as xgb

# Model save
import pickle

# ML Metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, plot_confusion_matrix, roc_curve, cohen_kappa_score
from sklearn.model_selection import GridSearchCV


# 1.0 Load Data


In [2]:
dscombined = pd.read_csv( 'C:/Users/PICHAU/Desktop/AIbox-Feedback/AIBOX-Mlfeedback/datasets/combined.csv' )
dsLIWC = pd.read_csv( 'C:/Users/PICHAU/Desktop/AIbox-Feedback/AIBOX-Mlfeedback/datasets/combinedLIWC.csv' )
dsCohmetrix = pd.read_csv( 'C:/Users/PICHAU/Desktop/AIbox-Feedback/AIBOX-Mlfeedback/datasets/combined_CohMetrix.csv' )


# 2.0 Merging data

In [3]:
dsCohmetrix = dsCohmetrix.rename( columns={ 'ID':'id' } )

In [4]:
mergedData = pd.merge(dsCohmetrix, dsLIWC, how='left', on='id')
mergedData = pd.merge(mergedData, dscombined, how='left', on='id')
# df1.merge(df2, how='inner', on='a')

In [5]:
mergedData

,id,cm.DESPC,cm.DESSC,cm.DESWC,cm.DESPL,cm.DESPLd,cm.DESSL,cm.DESSLd,cm.DESWLsy,cm.DESWLsyd,...,liwc.leisure,liwc.home,liwc.money,liwc.relig,liwc.death,liwc.assent,liwc.nonfl,liwc.filler,feedback,classe
0,23,1,1,18,1.0,0.0,17.000000,0.000000,1.888889,0.900254,...,1,0,0,0,0,0,0,0,Ler outros textos sobre o assunto para lher da...,0
1,24,1,2,24,2.0,0.0,10.500000,0.500000,1.916667,1.100066,...,1,0,1,0,0,0,0,0,Procure ler mais; para escrever mais acerca do...,0
2,35,1,2,31,2.0,0.0,12.500000,0.500000,1.741935,0.964978,...,2,0,0,0,0,0,0,0,Texto bom; precisa apenas ler mais um pouco; p...,0
3,37,1,1,7,1.0,0.0,6.000000,0.000000,1.857143,1.069045,...,0,0,0,0,0,0,0,0,Ter cuidado com copias da internet.,0
4,68,1,4,32,4.0,0.0,6.500000,3.640055,1.937500,1.268413,...,0,0,1,0,0,2,0,0,Carssimo; o trabalho precisa de uma sistematiz...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,13809,1,2,28,2.0,0.0,12.000000,11.000000,2.250000,1.404358,...,0,0,1,0,0,0,1,0,Rosivalda; timo texto; suas reflexes comparati...,1
996,13810,1,2,66,2.0,0.0,29.500000,27.500000,1.969697,1.149845,...,2,0,1,0,0,1,0,0,Ruth; bom texto sobre o filme e sobre as suas ...,1
997,13811,1,2,37,2.0,0.0,16.500000,15.500000,2.027027,1.213042,...,2,2,0,0,0,1,2,0,Silmara; excelente texto e leitura do filme co...,1
998,13813,1,3,28,3.0,0.0,7.666667,10.143416,1.964286,1.170063,...,2,0,0,1,0,0,1,0,Sueli; boa leitura do filme e da realidade; su...,1


## 2.1 Check Nan

In [6]:
mergedData.isna().sum().sort_values()


id                0
liwc.ipron        0
liwc.article      0
liwc.verb         0
liwc.auxverb      0
               ... 
cm.RDL2         165
cm.CRFCWO1      165
cm.CRFCWO1d     165
cm.LSASSpd      362
cm.LSASSp       362
Length: 161, dtype: int64

## 2.2 Fillout NAn

In [7]:
mergedData = mergedData.dropna(axis=1, how='any', inplace=False) # axis=1 => columns

In [8]:
mergedData.isna().sum().sort_values()

id              0
liwc.article    0
liwc.verb       0
liwc.auxverb    0
liwc.past       0
               ..
cm.SMCAUSr      0
cm.SMCAUSlsa    0
cm.SMCAUSwn     0
cm.CNCComp      0
classe          0
Length: 155, dtype: int64

# 3.0 Data Preparation



## 3.1 Normalization

In [9]:
# no analyzed variable looks like a normal distribution

## 3.2 Rescaling

In [10]:
mergedData.dtypes

id               int64
cm.DESPC         int64
cm.DESSC         int64
cm.DESWC         int64
cm.DESPL       float64
                ...   
liwc.death       int64
liwc.assent      int64
liwc.nonfl       int64
liwc.filler      int64
classe           int64
Length: 155, dtype: object

In [11]:
visu = mergedData.select_dtypes( exclude=['int64', 'float64','int32','UInt32'] )
print(visu)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[1000 rows x 0 columns]


In [12]:
# thus, due to feedback be a no-analyzed variable, merged dataset doesn't need rescaling
# so, the other variables doesnt need a changes, for now, because the outliers problem will be solved on cohmetrix software  

## 3.3 Transformation


In [13]:
# once time again, transformation is not necessary, due to mergedData only have feedback that is a categorical variable and the response variable is only 0 and 1

# 4.0 EDA - Correlation of numAtttributes

In [14]:
num_attributes = mergedData.select_dtypes( include=['int64', 'float64'] )

# 5.0 Feature Selection


In [15]:
# correlation = num_attributes.corr( method='pearson' )
# plt.figure( figsize=( 25, 19 ) )
# sns.heatmap( correlation, annot=True );

## 5.1 Split dataframe into training and test dataset

In [16]:
# training dataset
X_train = mergedData[mergedData['id']<12800] # this id split dataset in 3/4 proportion
y_train = X_train['classe']

# test dataset 
X_test = mergedData[mergedData['id']>=12800] # this id split dataset in 3/4 proportion
y_test = X_test['classe']


In [17]:
X_train

,id,cm.DESPC,cm.DESSC,cm.DESWC,cm.DESPL,cm.DESPLd,cm.DESSL,cm.DESSLd,cm.DESWLsy,cm.DESWLsyd,...,liwc.achieve,liwc.leisure,liwc.home,liwc.money,liwc.relig,liwc.death,liwc.assent,liwc.nonfl,liwc.filler,classe
0,23,1,1,18,1.0,0.0,17.000000,0.000000,1.888889,0.900254,...,0,1,0,0,0,0,0,0,0,0
1,24,1,2,24,2.0,0.0,10.500000,0.500000,1.916667,1.100066,...,1,1,0,1,0,0,0,0,0,0
2,35,1,2,31,2.0,0.0,12.500000,0.500000,1.741935,0.964978,...,4,2,0,0,0,0,0,0,0,0
3,37,1,1,7,1.0,0.0,6.000000,0.000000,1.857143,1.069045,...,0,0,0,0,0,0,0,0,0,0
4,68,1,4,32,4.0,0.0,6.500000,3.640055,1.937500,1.268413,...,2,0,0,1,0,0,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,12769,-1,-1,-1,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,...,0,0,0,0,0,0,0,0,0,0
757,12773,1,3,25,3.0,0.0,6.333333,5.557777,2.120000,1.394035,...,1,0,0,0,0,0,0,1,0,1
758,12788,1,2,13,2.0,0.0,5.500000,4.500000,2.230769,1.832750,...,0,0,0,0,0,0,0,0,0,1
759,12790,1,1,49,1.0,0.0,45.000000,0.000000,1.938776,1.375502,...,1,2,0,2,0,0,0,1,0,1


## 5.2 Boruta as Feature Selector

In [18]:
# training and test dataset for Boruta
X_train_n = X_train.drop( ['classe'], axis=1 ).values 
y_train_n = y_train.values.ravel() 

# define RandomForestRegressor
rf = RandomForestClassifier( random_state=42 )

# define Boruta
boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit( X_train_n, y_train_n )


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	154
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	154
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	154
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	154
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	154
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	154
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	154
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	24
Tentative: 	10
Rejected: 	120
Iteration: 	9 / 100
Confirmed: 	24
Tentative: 	10
Rejected: 	120
Iteration: 	10 / 100
Confirmed: 	24
Tentative: 	10
Rejected: 	120
Iteration: 	11 / 100
Confirmed: 	24
Tentative: 	10
Rejected: 	120
Iteration: 	12 / 100
Confirmed: 	24
Tentative: 	7
Rejected: 	123
Iteration: 	13 / 100
Confirmed: 	24
Tentative: 	7
Rejected: 	123
Iteration: 	14 / 100
Confirmed: 	24
Tentative: 	7
Rejected: 	123
Iteration: 	15 / 100
Confirmed: 	24
Tentative: 	7
Rejected: 	123
Iteration: 	16 / 100
Confirmed: 	24
T

### 4.2.1 Best features Boruta

In [19]:
cols_selected = boruta.support_.tolist() # devolve o valor do rank baseado na relevancia

# best features 
X_train_fs = X_train.drop( ['classe'], axis=1 ) # axis=1 dropar as colunas inteiras
cols_selected_boruta = X_train_fs.iloc[:, cols_selected].columns.to_list()

# # not selected boruta 
cols_not_selected_boruta = np.setdiff1d

In [20]:
del cols_selected_boruta[0] # removing 'id' from cols_selected_boruta 


In [21]:
cols_selected_boruta

['cm.DESWC',
 'cm.DESSL',
 'cm.DESWLsy',
 'cm.DESWLltd',
 'cm.SMCAUSlsa',
 'cm.SMCAUSwn',
 'cm.WRDNOUN',
 'cm.WRDVERB',
 'cm.WRDFRQc',
 'cm.WRDFRQa',
 'cm.WRDFRQmc',
 'cm.WRDFAMc',
 'cm.WRDCNCc',
 'cm.WRDIMGc',
 'cm.WRDMEAc',
 'cm.RDFRE',
 'cm.RDFKGL',
 'liwc.funct',
 'liwc.preps',
 'liwc.social',
 'liwc.cogmech',
 'liwc.relativ',
 'liwc.space']

In [22]:
cols_selected_boruta


['cm.DESWC',
 'cm.DESSL',
 'cm.DESWLsy',
 'cm.DESWLltd',
 'cm.SMCAUSlsa',
 'cm.SMCAUSwn',
 'cm.WRDNOUN',
 'cm.WRDVERB',
 'cm.WRDFRQc',
 'cm.WRDFRQa',
 'cm.WRDFRQmc',
 'cm.WRDFAMc',
 'cm.WRDCNCc',
 'cm.WRDIMGc',
 'cm.WRDMEAc',
 'cm.RDFRE',
 'cm.RDFKGL',
 'liwc.funct',
 'liwc.preps',
 'liwc.social',
 'liwc.cogmech',
 'liwc.relativ',
 'liwc.space']

# 6.0 Machine Learning Modelling

In [23]:
x_train = X_train[ cols_selected_boruta ]
x_test = X_test[ cols_selected_boruta ]

##  6.1. Stochastic Gradient Descent Classifier (SGDClassifier)

In [24]:
#  # Train SGDClassifier
# sgdc = SGDClassifier(random_state=42)
# sgdc.fit(X_train, y_train)

# # Predict
# sgdc_yhat = sgdc.predict(X_test)

# # ROC-AUC
# sgdc_cv = cross_val_predict(sgdc, X_train, y_train, cv=5, method='decision_function')
# sgd_roc_auc = roc_auc_score(y_train, sgdc_cv)

# # performance
# sgdc_performance = pd.DataFrame(['SGDClassifier',
#                                     accuracy_score(y_test, sgdc_yhat),
#                                     precision_score(y_test, sgdc_yhat),
#                                     recall_score(y_test, sgdc_yhat),
#                                     f1_score(y_test, sgdc_yhat),
#                                     sgd_roc_auc]).T
# sgdc_performance.columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']


# sgdc_performance

## 6.2 Logistic Regression

In [25]:
# Train Logistic Regression Model
logreg = LogisticRegression(random_state=42)
logreg.fit(x_train, y_train)

# Predict
logreg_yhat = logreg.predict(x_test)

# ROC-AUC
logreg_cv = cross_val_predict(logreg, x_train, y_train, cv=5, method='decision_function')
logreg_roc_auc = roc_auc_score(y_train, logreg_cv)

# performance
logreg_performance = pd.DataFrame(['Logistic Regression',
                                    accuracy_score(y_test, logreg_yhat),
                                    precision_score(y_test, logreg_yhat),
                                    recall_score(y_test, logreg_yhat),
                                    f1_score(y_test, logreg_yhat),
                                    cohen_kappa_score(y_test, logreg_yhat)]).T
logreg_performance.columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'cohen_kappa_score']


logreg_performance

,Model,Accuracy,Precision,Recall,F1-Score,cohen_kappa_score
0,Logistic Regression,0.947791,0.970213,0.974359,0.972281,0.524041


## 6.3 Random Forest Classifier

In [26]:
 # Train Random Forest
randforest = RandomForestClassifier(random_state=42, n_jobs=-1)
randforest.fit(x_train, y_train)

# Predict
randforest_yhat = randforest.predict(x_test)

# performance
randforest_performance = pd.DataFrame(['Random Forest Classifier',
                                    accuracy_score(y_test, randforest_yhat),
                                    precision_score(y_test, randforest_yhat),
                                    recall_score(y_test, randforest_yhat),
                                    f1_score(y_test, randforest_yhat),
                                    cohen_kappa_score(y_test, randforest_yhat)]).T
randforest_performance.columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'cohen_kappa_score']


randforest_performance

,Model,Accuracy,Precision,Recall,F1-Score,cohen_kappa_score
0,Random Forest Classifier,0.943775,0.970085,0.970085,0.970085,0.503419


## 6.4 Support Vector Machine Classifier

In [27]:
# Training SVM
svm = SVC(random_state=42)
svm.fit(x_train, y_train)

# Predict
svm_yhat = svm.predict(x_test)

# performance
svm_performance = pd.DataFrame( ['SVM Classifier',
                                    accuracy_score(y_test, svm_yhat),
                                    precision_score(y_test, svm_yhat),
                                    recall_score(y_test, svm_yhat),
                                    f1_score(y_test, svm_yhat),
                                    cohen_kappa_score(y_test, svm_yhat)] ).T
svm_performance.columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'cohen_kappa-score']


svm_performance

,Model,Accuracy,Precision,Recall,F1-Score,cohen_kappa-score
0,SVM Classifier,0.955823,0.955102,1.0,0.977035,0.405986


## 6.5 XGB Classifier

In [28]:
 # Train Random Forest
xgboost = xgb.XGBClassifier(random_state=42, eval_metric='error')
xgboost.fit(x_train, y_train);

# Predict
xgboost_yhat = xgboost.predict(x_test);

# performance
xgboost_performance = pd.DataFrame(['XGBooster Classifier',
                                    accuracy_score(y_test, xgboost_yhat),
                                    precision_score(y_test, xgboost_yhat),
                                    recall_score(y_test, xgboost_yhat),
                                    f1_score(y_test, xgboost_yhat),
                                    cohen_kappa_score(y_test, xgboost_yhat) ]).T
xgboost_performance.columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'cohen_kappa_score']

xgboost_performance

,Model,Accuracy,Precision,Recall,F1-Score,cohen_kappa_score
0,XGBooster Classifier,0.935743,0.973913,0.957265,0.965517,0.495441


## 6.6 Cross-validation of all models

In [29]:
# Merging train and test dataset to make the Cross-Validation
x_full = pd.concat([x_train, x_test])
y_full = pd.concat([y_train, y_test])

In [30]:
# Accuracy of randforest
randforest_cv = cross_val_score(randforest, x_full, y_full, cv=10, scoring='accuracy')
randforest_cv_acc = pd.DataFrame(['Randforest', (randforest_cv.mean() * 100).round(3),(randforest_cv.std() * 100).round(3)]).T

# # Accuracy of sdgc
# sgdc_cv = cross_val_score(sgdc, x_full, y_full, cv=10, scoring='accuracy')
# sgdc_cv_acc = pd.DataFrame(['Sdgc', (sgdc_cv.mean() * 100).round(3),(sgdc_cv.std() * 100).round(3)]).T

# Accuracy of logistic Regression
logreg_cv = cross_val_score(logreg, x_full, y_full, cv=10, scoring='accuracy')
logreg_cv_acc = pd.DataFrame(['logistic Reg', (logreg_cv.mean() * 100).round(3),(logreg_cv.std() * 100).round(3)]).T

# Accuracy of SVM Model
svm_cv = cross_val_score(svm, x_full, y_full, cv=10, scoring='accuracy')
svm_cv_acc = pd.DataFrame(['SVM classifier', (svm_cv.mean() * 100).round(3),(svm_cv.std() * 100).round(3)]).T

# Accuracy of XGBoost Classifier
xgboost_cv = cross_val_score(xgboost, x_full, y_full, cv=10, scoring='accuracy')
xgboost_cv_acc = pd.DataFrame(['XGBoost Classifier', (xgboost_cv.mean() * 100).round(3),(xgboost_cv.std() * 100).round(3)]).T



General_cv= pd.concat([randforest_cv_acc, logreg_cv_acc, svm_cv_acc, xgboost_cv_acc])

General_cv.columns = ['Model', 'Mean', 'std']
General_cv

,Model,Mean,std
0,Randforest,92.6,3.583
0,logistic Reg,92.1,2.982
0,SVM classifier,91.7,2.193
0,XGBoost Classifier,91.7,3.318


# 7.0 Hyperparameter Fine Tuning

## 7.1 Grid Search

In [31]:
# Number of trees in random forest
n_estimators_k = [int(x) for x in np.linspace(start = 50, stop = 1000, num = 20)]
# Maximum number of levels in tree
max_depth_k = [int(y) for y in np.linspace(start = 1, stop = 50, num = 50)]

        

In [32]:
# Create the param_grid
param_grid = {'n_estimators': n_estimators_k,
               'max_depth': max_depth_k
             }

In [33]:
rf_Grid = GridSearchCV(estimator = randforest, param_grid = param_grid, verbose=2, n_jobs = 4)


In [34]:
rf_Grid.fit(x_train, y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             n_jobs=4,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                       23, 24, 25, 26, 27, 28, 29, 30, ...],
                         'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400,
                                          450, 500, 550, 600, 650, 700, 750,
                                          800, 850, 900, 950, 1000]},
             verbose=2)

In [36]:
rf_Grid.best_params_


{'max_depth': 6, 'n_estimators': 300}

## 7.2 Final Model

In [37]:
param_tuned = {
    'n_estimators': 300,
    'max_depth': 6,
        }

In [51]:
 # Train Random Forest
randforest_tuned = RandomForestClassifier(max_depth=param_tuned['max_depth'], n_estimators=param_tuned['n_estimators'], random_state=43). fit(x_train, y_train)
randforest_tuned.fit(x_train, y_train)

# Predict
randforest_tuned_yhat = randforest_tuned.predict( x_test )


# performance
randforest_tuned_performance = pd.DataFrame(['Random Forest Classifier',
                                    accuracy_score(y_test, randforest_tuned_yhat),
                                    precision_score(y_test, randforest_tuned_yhat),
                                    recall_score(y_test, randforest_tuned_yhat),
                                    f1_score(y_test, randforest_tuned_yhat),
                                    cohen_kappa_score(y_test, randforest_tuned_yhat)]).T
randforest_tuned_performance.columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'cohen_kappa_score']


randforest_tuned_performance

,Model,Accuracy,Precision,Recall,F1-Score,cohen_kappa_score
0,Random Forest Classifier,0.951807,0.974359,0.974359,0.974359,0.574359


In [58]:
pickle.dump( randforest_tuned, open('C:/Users/PICHAU/Desktop/AIbox-Feedback/AIBOX-Mlfeedback/param_and_model//model_feedback.pkl', 'wb' ) )